# Inference 
- Perform inference on TCGA test set using pre-trained model trained on mouse data (use the weights saved in '../code/checkpoints').
- Test notebook for script.
- Script generates `0107_pred_script_TCGA.csv`.

#### Notes
- Run pre-trained model on tile test set.
- Test set contains all WSIs except those used for training.

***
## Inference (test code for script)

In [1]:
import torch

In [2]:
'Load metadata'
data_fld = '../data'
sub_fld = os.path.join(data_fld, 'TCGA_data')
path = os.path.join(sub_fld, 'tcga_wsi_meta.csv')
df_wsi = pd.read_csv(path, index_col=0)
path = os.path.join(sub_fld, 'tcga_tile_meta.csv')
df_tile = pd.read_csv(path, index_col=0)

In [3]:
df_wsi.head(2)

,wsi_name,wsi_id,clinical_sample_id,primary_tumor_type,CNA_data,ABSOLUTE_purity,rna_subtype,BetaCAT,P53,PTEN,APC
clinical_donor_id,,,,,,,,,,,
TCGA-3N-A9WB,TCGA-3N-A9WB-01Z-00-DX1.A9950ED4-9480-455C-AE0...,TCGA-3N-A9WB-01Z-00-DX1.svs,TCGA-3N-A9WB-06,ACRAL/NON-CUTANEOUS,True,0.92,Common,0,0,0,0
TCGA-3N-A9WC,TCGA-3N-A9WC-01Z-00-DX1.C833FCAB-6329-4F90-88E...,TCGA-3N-A9WC-01Z-00-DX1.svs,TCGA-3N-A9WC-06,NON-ACRAL CUTANEOUS,True,0.55,Common,0,0,0,1


In [4]:
df_tile.head(2)

,x_tile_coord,y_tile_coord,clinical_donor_id,wsi_name,clinical_sample_id,primary_tumor_type,CNA_data,ABSOLUTE_purity,rna_subtype,BetaCAT,P53,PTEN,APC
wsi_id,,,,,,,,,,,,,
TCGA-D3-A8GO-06Z-00-DX1.svs,4,8,TCGA-D3-A8GO,TCGA-D3-A8GO-06Z-00-DX1.357CD90F-23D3-45BB-BA1...,TCGA-D3-A8GO-06,NaN,True,0.37,Common,0,0,1,0
TCGA-D9-A4Z5-01Z-00-DX1.svs,7,1,TCGA-D9-A4Z5,TCGA-D9-A4Z5-01Z-00-DX1.88AC8735-B520-4FCE-BC0...,TCGA-D9-A4Z5-01,ACRAL/NON-CUTANEOUS,True,0.93,Common,0,0,0,0


In importing this model check that:
- it loads the ORIGINAL weights of the patchwise network (`/weights_pw1.pth` even though it says it fails).
- it loads the SAURABH weights for the image wise network (`'/weights_iw_h2gnew_trn1.pth`)

In [11]:
from src import *
import sys
sys.argv[1:] = []
args = ModelOptions().parse()


------------ Options -------------
batch_size: 64
beta1: 0.9
beta2: 0.999
channels: 1
checkpoints_path: ./checkpoints
cuda: True
dataset_path: /home/tbiancal/melanoma_images/2021_1_7_set
debug: False
ensemble: 1
epochs: 10
gpu_ids: 0
log_interval: 5
lr: 0.001
network: 0
no_cuda: False
patch_stride: 256
seed: 1
test_batch_size: 64
testset_path: ./home/tbiancal/melanoma_images/2021_1_7_set/test
-------------- End ----------------



In [12]:
pw_network = PatchWiseNetwork(args.channels)
iw_network = ImageWiseNetwork(args.channels)
im_model = ImageWiseModel(args, iw_network, pw_network)

Loading "image-wise" model...
Loading "patch-wise" model...
Failed to load pre-trained network from  ./checkpoints/weights_pw1.pth


In [13]:
img_fld = os.path.join(home, 'melanoma_images/TCGA_tiles')
# os.listdir(img_fld)

In [14]:
df_out = df_tile.reset_index().sample(n=10)

In [ ]:
preds = []
for ix, entry in tqdm(df_out.iterrows()):
    file = entry.wsi_id.split('.svs')[0] + '_' + str(entry.x_tile_coord) + '_' + str(entry.y_tile_coord) + '.png'
    tile_path = os.path.join(img_fld, file)
    assert os.path.isfile(tile_path)
    [[pred, conf, file]], output = im_model.test(tile_path, ensemble=args.ensemble == 1, verbose=False)
    softmax_res = torch.sum(np.exp(output.cpu().data), dim=0)
    probs = np.array(softmax_res / torch.sum(softmax_res))
    preds.append(probs)

1it [00:01,  1.40s/it]

In [60]:
df_out['pred_prob'] = preds

In [61]:
df_out

,wsi_id,x_tile_coord,y_tile_coord,clinical_donor_id,wsi_name,clinical_sample_id,primary_tumor_type,CNA_data,ABSOLUTE_purity,rna_subtype,BetaCAT,P53,PTEN,APC,pred_prob
17144,TCGA-EE-A2MS-06Z-00-DX1.svs,2,4,TCGA-EE-A2MS,TCGA-EE-A2MS-06Z-00-DX1.76FF55CE-10D0-4A93-AB1...,TCGA-EE-A2MS-06,NON-ACRAL CUTANEOUS,True,0.81,Common,0,0,0,0,"[0.4250285, 0.05235385, 0.15658085, 0.36019465..."
5738,TCGA-ER-A2NB-01Z-00-DX1.svs,3,1,TCGA-ER-A2NB,TCGA-ER-A2NB-01Z-00-DX1.323F02C6-D07A-41B6-BEE...,TCGA-ER-A2NB-01,ACRAL/NON-CUTANEOUS,True,0.47,OxPhos,0,0,0,0,"[2.803946e-06, 0.88176084, 1.51564e-06, 3.1782..."
19384,TCGA-D3-A51K-06Z-00-DX1.svs,5,4,TCGA-D3-A51K,TCGA-D3-A51K-06Z-00-DX1.CBDB017A-31B3-430E-B69...,TCGA-D3-A51K-06,NON-ACRAL CUTANEOUS,True,0.85,OxPhos,0,0,0,0,"[0.3872154, 0.25397992, 0.043275878, 0.0612559..."
21769,TCGA-ER-A3ET-06Z-00-DX1.svs,6,1,TCGA-ER-A3ET,TCGA-ER-A3ET-06Z-00-DX1.8EBAFF1A-E8FE-446A-8A1...,TCGA-ER-A3ET-06,NON-ACRAL CUTANEOUS,True,0.60,Common,0,1,0,0,"[0.3238556, 0.017040828, 0.30280724, 0.1369792..."
7675,TCGA-XV-AAZY-01Z-00-DX1.svs,8,6,TCGA-XV-AAZY,TCGA-XV-AAZY-01Z-00-DX1.77D5F2CA-1A90-4F8B-86A...,TCGA-XV-AAZY-01,NON-ACRAL CUTANEOUS,True,0.46,OxPhos,0,0,0,0,"[1.2950848e-12, 2.4051384e-05, 1.0649153e-11, ..."
14777,TCGA-D3-A1Q9-06Z-00-DX1.svs,3,4,TCGA-D3-A1Q9,TCGA-D3-A1Q9-06Z-00-DX1.DBECA4A2-6BCF-4EA3-91F...,TCGA-D3-A1Q9-06,NON-ACRAL CUTANEOUS,True,0.69,Common,0,0,0,0,"[3.563653e-05, 0.16989492, 2.810121e-07, 1.123..."
18969,TCGA-EE-A2MS-06Z-00-DX1.svs,3,8,TCGA-EE-A2MS,TCGA-EE-A2MS-06Z-00-DX1.76FF55CE-10D0-4A93-AB1...,TCGA-EE-A2MS-06,NON-ACRAL CUTANEOUS,True,0.81,Common,0,0,0,0,"[0.57511985, 0.017666735, 0.38219076, 0.023588..."
3677,TCGA-GF-A6C8-06Z-00-DX1.svs,9,5,TCGA-GF-A6C8,TCGA-GF-A6C8-06Z-00-DX1.9388CB1A-BF64-4CE8-AAC...,TCGA-GF-A6C8-06,NON-ACRAL CUTANEOUS,True,0.68,Common,0,1,0,0,"[0.13743345, 0.58786386, 0.0009981238, 0.26102..."
17721,TCGA-BF-A1PU-01Z-00-DX1.svs,2,2,TCGA-BF-A1PU,TCGA-BF-A1PU-01Z-00-DX1.CB0A52E3-16A9-46B2-BBE...,TCGA-BF-A1PU-01,NON-ACRAL CUTANEOUS,True,0.90,OxPhos,0,0,1,0,"[1.1163783e-11, 0.00012110643, 1.7551815e-06, ..."
8880,TCGA-D3-A3MO-06Z-00-DX1.svs,2,3,TCGA-D3-A3MO,TCGA-D3-A3MO-06Z-00-DX1.F375C5A1-723B-4F61-B1A...,TCGA-D3-A3MO-06,NON-ACRAL CUTANEOUS,True,0.92,MITF-Low,0,0,1,0,"[0.46628955, 0.5337026, 2.877063e-06, 4.128896..."


In [54]:
# ne salvo uno per fare test veloci
df_out.to_csv('../results/0107_pred.csv')